In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    RocCurveDisplay
)

sns.set(style="whitegrid")

In [ ]:
df = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df["HeartDisease"].value_counts(normalize=True)

In [ ]:
X = df.drop(columns="HeartDisease")
y = df["HeartDisease"]

In [ ]:
fig, ax = plt.subplots()
ax = sns.countplot(data=df, x="ChestPainType", hue="HeartDisease")
plt.xticks(rotation=30)
plt.title("Chesst Pain vs Heart Disease")
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data=df[numeric_features].corr(), cmap="coolwarm", center=0)
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

X_train.columns.tolist()

In [ ]:
numeric_features = df.select_dtypes(include="number").columns.tolist()
numeric_features.remove("HeartDisease")

categorical_features = df.select_dtypes(include="object").columns.tolist()

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_features)
    ]
)

In [ ]:
pipeline = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", LogisticRegression(
            max_iter=1000,
            class_weight="balanced",
            random_state=42
        ))
    ]
)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:,1]

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(data=conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
roc_auc_score(y_test, y_proba)

In [ ]:
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.show()

In [ ]:
cv_scores = cross_val_score(
    pipeline,
    X_train,
    y_train,
    cv=5,
    scoring="roc_auc"
)
cv_scores, cv_scores.mean()

In [ ]:
feature_names = (
    numeric_features +
    pipeline.named_steps["preprocessing"]
    .named_transformers_["cat"]
    .get_feature_names_out(categorical_features)
    .tolist()
)

coefficients = pipeline.named_steps["model"].coef_[0]

coef_df = (
    pd.DataFrame({
        "feature": feature_names,
        "coefficient": coefficients
    })
    .sort_values(by="coefficient", key=abs, ascending=False)
)

coef_df.head(15)